In [1]:
# !pip install Folium

In [2]:
# 지도를 활용 : Folium 라이브러리
import folium
import pandas as pd

In [3]:
# 서울의 지도 만들기
seoul_map = folium.Map(
    location=[37.55,126.98],
    zoom_start=12,
    control_scale=True,
)
# location이 잘못되면 세계지도크기로 가져온다. 오류발생안함

# 파일로 저장
seoul_map.save('./output/seoul.html')

In [4]:
# 지도 스타일 적용하기
seoul_map2 = folium.Map(location=[37.55,126.98],zoom_start=12,
                        tiles='Stamen Terrain')
seoul_map3 = folium.Map(location=[37.55,126.98],zoom_start=12,
                        tiles='Stamen Toner')

seoul_map2.save('./output/seoul2.html')
seoul_map3.save('./output/seoul3.html')

In [5]:
# 지도에 마커 표시하기
# 위도 경도의 자료가 있는 데이터를 load
df = pd.read_excel('./dataset/서울지역 대학교 위치.xlsx',index_col=0)

seoul_map = folium.Map(location=[37.55,126.98],zoom_start=12,
                        tiles='Stamen Terrain')

print(df.head())

for name, lat,lng in zip(df.index,df.위도,df.경도):
#     print(name, lat, lng)
    folium.Marker([lat,lng],popup=name).add_to(seoul_map)

seoul_map.save('./output/seoul_c1.html')

                     위도          경도
KAIST 서울캠퍼스   37.592573  127.046737
KC대학교         37.548345  126.854797
가톨릭대학교(성신교정)  37.585922  127.004328
가톨릭대학교(성의교정)  37.499623  127.006065
감리교신학대학교      37.567645  126.961610


In [6]:
seoul_map = folium.Map(location=[37.55,126.98],zoom_start=12,
                        tiles='Stamen Terrain')

for name, lat,lng in zip(df.index,df.위도,df.경도):
#     print(name, lat, lng)
    folium.CircleMarker([lat,lng],popup=name,
                 radius=5,color='red',fill=True,fill_color='yellow',
                 fill_opacity=0.7, # 투명도
                 ).add_to(seoul_map)

seoul_map.save('./output/seoul_c2.html')

In [7]:
# 지도 영역에 단계구분도(Choropleth) 표시
import json

df = pd.read_excel('./dataset/경기도인구데이터.xlsx',index_col='구분')
print(df.head())
print(df.columns)
df.columns = df.columns.map(str)
print(df.columns)

          2007    2008    2009    2010    2011    2012    2013    2014  \
구분                                                                       
수원시장안구  287474  285803  290798  293692  290999  291757  300908  301196   
수원시권선구  310553  308177  304097  306783  321176  332633  331773  339835   
수원시팔달구  216945  213235  219833  216503  209772  206794  204805  203479   
수원시영통구  252730  260210  258421  260557  266542  289074  310671  329718   
성남시수정구  256744  248452  242161  241070  236123  233997  228167  223539   

          2015    2016    2017  
구분                              
수원시장안구  299016  296479  293828  
수원시권선구  351053  358393  365653  
수원시팔달구  199180  198515  193311  
수원시영통구  335375  340654  349836  
성남시수정구  219531  232841  236932  
Int64Index([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype='int64')
Index(['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017'],
      dtype='object')


In [8]:
# 경기도 시군구 경계 정보를 가져옴
geo_path = './dataset/경기도행정구역경계.json'

try:
    geo_data = json.load(open(geo_path,encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path,encoding='utf-8-sig'))
    
geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': 31380,
    'name': '양평군',
    'name_eng': 'Yangpyeong-gun',
    'base_year': 2013},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.56113535909492, 37.6256560539258],
      [127.57290963929243, 37.61087369350498],
      [127.56366973908277, 37.5842624297699],
      [127.52226606559191, 37.5828628697518],
      [127.50048767007603, 37.569036373178626],
      [127.47687473909174, 37.574448241913856],
      [127.47565866803647, 37.60512112601634],
      [127.44699730711024, 37.64497584741164],
      [127.4272349102434, 37.66284420898682],
      [127.40156700708162, 37.64806970784708],
      [127.3732642199939, 37.6453987554284],
      [127.3542931884407, 37.6250006376975],
      [127.34360057873045, 37.588997440929354],
      [127.31002843450217, 37.53523876142835],
      [127.30923294884336, 37.5135706079458],
      [127.32809611134748, 37.53133849534739],
      [127.3663244453597, 37.5

In [9]:
g_map = folium.Map(location=[37.55,126.98],zoom_start=9,
                        tiles='Stamen Terrain')

year = '2017'

# 경계구분을 추가
cp = folium.Choropleth(geo_data=geo_data, # 지도 경계 데이터
                  data=df[year], # 표시할 데이터
                  columns=[df.index,df[year]], # 열 지정
                  fill_color='YlOrRd',fill_opacity=0.7,line_opacity=0.5,
                  threshold_scale=[10000,100000,300000,500000,700000],
                  key_on = 'feature.properties.name',
                 ).add_to(g_map)

# 마우스 오버레이시 구역별 이름 보여주기 GeoJsonTooltip
folium.LayerControl().add_to(g_map)
cp.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)

g_map.save('./output/g_map'+year+'.html')

In [10]:
outputg_map = folium.Map(location=[37.5502,126.982], 
                   tiles='Stamen Terrain', zoom_start=9)

# 출력할 연도 선택 (2007 ~ 2017년 중에서 선택)
year = '2017'  

# Choropleth 클래스로 단계구분도 표시하기
fmap=folium.Choropleth(geo_data=geo_data,    # 지도 경계
                 data = df[year],      # 표시하려는 데이터
                 columns = [df.index, df[year]],  # 열 지정
                 fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
                 threshold_scale=[10000, 100000, 300000, 500000, 700000],               
                 key_on='feature.properties.name',
                 highlight=True
                 
                 ).add_to(g_map)

# 클릭 시 구역 이름 출력 GeooJsonPopup
fmap.geojson.zoom_on_click = False
fmap.geojson.add_child(
    folium.features.GeoJsonPopup(['name'],labels=False) # Popup
)

display(g_map)
# 지도를 HTML 파일로 저장하기
g_map.save('output/gyonggi_population_' + year + '_3.html')

In [11]:
# 마커 툴팁추가, 팝업 텍스트포맷

seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                       zoom_start=12)
tooltip = '클릭하셔용!'

folium.Marker([37.55,126.98], popup='<div style="width:100px;color:green;text-align:center">남산</div>', tooltip=tooltip).add_to(seoul_map)
folium.Marker([37.58,126.975], popup='<div style="width:100px;text-align:center"><b>경복궁</b></div>', tooltip=tooltip).add_to(seoul_map)
display(seoul_map)

In [12]:
import pandas as pd

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

In [17]:
m = folium.Map(location=[48, -102], zoom_start=3)

fmap = folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

# 툴팁처리 추가
fmap.geojson.zoom_on_click = False
fmap.geojson.add_child(
     folium.features.GeoJsonTooltip(['name'],labels=False) # Tooltip
)

m

In [19]:
m = folium.Map(location=[48, -102], zoom_start=3)

fmap = folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

# 팝업처리 추가
fmap.geojson.zoom_on_click = False
fmap.geojson.add_child(
     folium.features.GeoJsonPopup(['name'],labels=False) # Popup
)

m